In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.6 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=c3d9aa8bfd0925050e301bdb9c8e1b3271e2177b67f7a97746631d7885041b2e
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM

import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')
from sentence_transformers import SentenceTransformer

model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')
#sentence = "In this way , the <mask> can change shape without <mask>."
s=['One', 'such', 'place', 'favourable', 'with', 'birds', 'is', 'the', '<mask>', 'of', '<mask>']
sentence=' '.join(s)
def get_prediction (sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')

    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    mask_index_guess=dict()
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=10, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
        print ("Mask ",index+1,"Guesses : ",words)
        mask_index_guess[mask_index]=words
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess,mask_index_guess


print ("Original Sentence: ",sentence)
sentence = sentence.replace("___","<mask>")
print ("Original Sentence replaced with mask: ",sentence)
print ("\n")

predicted_blanks,index = get_prediction(sentence)
print ("\nBest guess for fill in the blank :::",predicted_blanks)

Original Sentence:  One such place favourable with birds is the <mask> of <mask>
Original Sentence replaced with mask:  One such place favourable with birds is the <mask> of <mask>


Mask  1 Guesses :  ['Isle', 'University', 'Forest', 'Gulf', 'island', 'Island', 'House', 'Bay', 'city', 'town']
Mask  2 Guesses :  ['Maine', 'Florida', 'Gibraltar', 'Newfoundland', 'Edinburgh', 'Scotland', 'Bern', 'Dover', 'Cambridge', 'Aberdeen']

Best guess for fill in the blank :::  Isle Maine


In [ ]:
import scipy
def sort_by_similarity(original_sentence,generated_sentences_list):
    # Each sentence is encoded as a 1-D vector with 768 columns
    sentence_embeddings = model_BERT.encode(generated_sentences_list)

    queries = [original_sentence]
    query_embeddings = model_BERT.encode(queries)
    # Find the top sentences of the corpus for each query sentence based on cosine similarity
    number_top_matches = len(generated_sentences_list)

    dissimilar_sentences = []
    similar_sentences=[]

    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])


        for idx, distance in reversed(results[0:number_top_matches]):
            score = 1-distance
            if score < 0.96:
                dissimilar_sentences.append(generated_sentences_list[idx].strip())
            else:
                similar_sentences.append(generated_sentences_list[idx].strip())
    sorted_dissimilar_sentences = sorted(dissimilar_sentences, key=len)
    sorted_similar_sentences = sorted(similar_sentences, key=len)
    return sorted_dissimilar_sentences,sorted_similar_sentences

In [ ]:

from nltk import tokenize
import nltk
nltk.download('punkt')

def generate_false_sentences(sentence,maskindices,original_mask_index,masked_tokens):
  generated_sentences=[]
  tokenized=tokenize.sent_tokenize(sentence)
  a=[]
  for sent in tokenized:
    a.extend(sent.split())

  original_mask_index=list(sorted(original_mask_index))
  generate=True
  while generate:
    mask_index=[key for key in maskindices.keys()]
    mask1=mask_index[0]
    for val1 in maskindices[mask1]:
      if (val1 in masked_tokens) or (len(val1)<=2):
        continue
      a[original_mask_index[0]]=val1
      try:
        mask2=mask_index[1]
        for val2 in maskindices[mask2]:
          if (val2 in masked_tokens) or (len(val2)<=2):
            continue
          a[original_mask_index[1]]=val2
          #print(a)
          false_statement=' '.join(a)
          #print(false_statement)
          generated_sentences.append(false_statement)

      except:
        false_statement=' '.join(a)
        #print(false_statement)
        generated_sentences.append(false_statement)
    generate=False  
    return generated_sentences  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
original="One such place favourable with birds is the Rann of Kuchch"
dissimilar,similar=sort_by_similarity(original,generated_sentences)

NameError: ignored

In [ ]:
print(similar)
print(dissimilar)

['One such place favourable with birds is the Bay of Dover', 'One such place favourable with birds is the Gulf of Dover', 'One such place favourable with birds is the Isle of Dover', 'One such place favourable with birds is the House of Dover', 'One such place favourable with birds is the Island of Dover', 'One such place favourable with birds is the island of Dover', 'One such place favourable with birds is the Bay of Gibraltar', 'One such place favourable with birds is the Isle of Gibraltar', 'One such place favourable with birds is the House of Aberdeen']
['One such place favourable with birds is the Bay of Bern', 'One such place favourable with birds is the Bay of Maine', 'One such place favourable with birds is the city of Bern', 'One such place favourable with birds is the town of Bern', 'One such place favourable with birds is the Gulf of Bern', 'One such place favourable with birds is the Isle of Bern', 'One such place favourable with birds is the Gulf of Maine', 'One such plac

In [ ]:
import json
NUM_MASK=5
with open("/content/gdrive/MyDrive/FYP/Answer Identification/Answerpredictions.json",'r') as f:
  Answer_Predictions=json.load(f)



In [ ]:
print(type(Answer_Predictions))

<class 'list'>


In [10]:
#generated_false_statements=list()
generated_questions=list()
for i,_ in enumerate(Answer_Predictions):
  data=dict()
  generated_false_statements=list()
  data=Answer_Predictions[i]
  original_sentence=Answer_Predictions[i]["Statement"].strip("\n")
  print(original_sentence)

  mask_indices=Answer_Predictions[i]["Mask_index"]
  mask_token=Answer_Predictions[i]["Mask_Token"]
  tokens=Answer_Predictions[i]["Tokens"]
  mask_portion=get_masks(tokens,mask_indices)
  #print(mask_portion)
  raw_sentence=[Answer_Predictions[i]["Tokens"]]*len(mask_portion)
  #print(raw_sentence)
  for Index,indices in enumerate(mask_portion):
    #print(indices)
    #print(Index)
    sentence=' '.join(raw_sentence[Index])
    sentence=sentence.split()
    for i in indices:
      sentence[i]="<mask>"
    #print(raw_sentence[Index])
    #print(sentence)
    sentence=" ".join(sentence)

    pred,ind=get_prediction(sentence)
    generated=generate_false_sentences(sentence,ind,indices,mask_token)
    if len(generated)<5:
      continue
    dissimilar,similar=sort_by_similarity(original_sentence,generated)
    #print(dissimilar)
    #print(similar[:10])
    if len(dissimilar)<=5:
      generated_false_statements.extend(dissimilar)
    else:
      generated_false_statements.extend(dissimilar[0:7])
    if len(similar)<=5:
      generated_false_statements.extend(similar)
    else:
      generated_false_statements.extend(similar[0:7])
    
  data["Generated False Question"]=list(set(generated_false_statements))
    #print(generated_false_statements)
    #print(data)
  generated_questions.append(data)


print(generated_questions[2])
with open("/content/gdrive/MyDrive/FYP/False Statement Generation/Generated_Questions.json","w") as f:
  json.dump(generated_questions,f)

Streaming output truncated to the last 5000 lines.
Mask  1 Guesses :  ['outcome', 'accumulation', 'effect', 'evolution', 'accident', 'artifact', 'adaptation', 'act', 'explosion', 'impact']
Mask  2 Guesses :  ['allow', 'enable', 'allowed', 'enabled', 'allows', 'helped', 'help', 'enables', 'caused', 'cause']
 Instead, we look for similarities among the organisms, which will allow we to put all these living organisms into different classes and then study different classes or groups as a whole
Mask  1 Guesses :  ['in', 'between', 'among', 'across', 'within', 'to', 'amongst', 'throughout', 'of', 'with']
Mask  2 Guesses :  ['species', 'organisms', 'groups', 'genomes', 'genome', 'organism', 'animals', 'group', 'biology', 'models']
Mask  1 Guesses :  ['differences', 'relationships', 'similarities', 'interactions', 'distinctions', 'patterns', 'correlations', 'connections', 'similarity', 'relations']
Mask  2 Guesses :  ['organisms', 'species', 'groups', 'animals', 'cells', 'classes', 'bacteria',

In [ ]:
Answer_Predictions[0]["Generated False Question"]

KeyError: ignored

In [ ]:
from nltk.corpus import stopwords
import random
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))
def get_masks(tokens,mask_index,k=2):
  stop_words_idx=[idx for idx,token in enumerate(tokens) if token in stop_words]
  mask_indices=[id for id in mask_index if id not in stop_words_idx]
  masks=list()
  if len(mask_indices)<=k:
    for i in mask_indices:
      masks.append([i])
  else:
    it=1
    for it in range(NUM_MASK):
      sample=random.sample(mask_indices,k=k)
      if sample not in masks:
        #print("h")
        add_sample=True
        #for i in sample:
        #  if not (tokens[i] in stop_words):
        #    add_sample=True
        #  else:
        #    #print(tokens[i])
        #    add_sample=False
        if add_sample:
          #print("added")
          masks.append(sample)
          #it=it+1
  return list(masks)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['O', 'O', 'O', 'O', 'time', 'for', 'trees', 'to', 'shed', 'trees', 'leaves.The', 'soils', 'also', 'vary', 'over', 'space']
['O', 'O', 'O', 'O', 'time', 'for', 'trees', 'to', 'shed', 'trees', 'leaves.The', 'soils', 'also', 'vary', 'over', 'space']
['O', 'O', 'O', 'O', 'time', 'for', 'trees', 'to', 'shed', 'trees', 'leaves.The', 'soils', 'also', 'vary', 'over', 'space']
['O', 'O', 'O', 'O', 'time', 'for', 'trees', 'to', 'shed', 'trees', 'leaves.The', 'soils', 'also', 'vary', 'over', 'space']
['O', 'O', 'O', 'O', 'time', 'for', 'trees', 'to', 'shed', 'trees', 'leaves.The', 'soils', 'also', 'vary', 'over', 'space']
['O', 'O', 'O', 'O', 'time', 'for', 'trees', 'to', 'shed', 'trees', 'leaves.The', 'soils', 'also', 'vary', 'over', 'space']
['O', 'O', 'O', 'O', 'time', 'for', 'trees', 'to', 'shed', 'trees', 'leaves.The', 'soils', 'also', 'vary', 'over', 'space']
['O', 'O', 'O', 'O', 'time', 'for', 'trees', 'to', 'shed', 'trees', 'leaves.The', 'soils', 'also', 'vary', 'over', 'space']
['O', 'O

[0, 1]


In [ ]:
#!/usr/bin/env python
# coding: utf-8

# import language_check
import string 
import nltk
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import framenet as fn
from nltk import pos_tag
from nltk.corpus import stopwords
# from stanfordcorenlp import StanfordCoreNLP
import json
import math
import requests

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('framenet_v17')
nltk.download('omw-1.4')

def find_related_terms(term, weight=0):
    # find synonyms and antonyms of a given term from ConceptNet.
    # weight value says how believable the information is. A typical weight is 1.0, and the number is higher when the information comes from more sources or more reliable sources. 
    set_synonym = set()
    set_antonym = set()
    set_related = set()
    
    term = term.strip().replace(" ","_")
    obj_synonym = requests.get(f'http://api.conceptnet.io/query?node=/c/en/{term}&rel=/r/Synonym&other=/c/en').json()
    obj_antonym = requests.get(f'http://api.conceptnet.io/query?node=/c/en/{term}&rel=/r/Antonym&other=/c/en').json()
    obj_related = requests.get(f'http://api.conceptnet.io/query?node=/c/en/{term}&rel=/r/RelatedTo&other=/c/en').json()
    
    def update_terms(obj, set_terms):
        for edge in obj['edges']:
            if edge['weight'] >= weight:
                start_term = edge['start']['term'].split('/')[3]
                if start_term != term:
                    set_terms.update(start_term.replace("_"," ").split())
                end_term = edge['end']['term'].split('/')[3]
                if end_term != term:
                    set_terms.update(end_term.replace("_"," ").split())
            
    update_terms(obj_synonym, set_synonym)
    update_terms(obj_antonym, set_antonym)
    update_terms(obj_related, set_related)
    
#     print(f"Synonym set:{set_synonym}")
#     print(f"Antonym set:{set_antonym}")
#     print(f"Related term set:{set_related}")
    return set_synonym | set_antonym | set_related


# create a set of stopwords (lemmatized)
stop_words = set(stopwords.words('english'))  

def get_wordnet_pos(word):
    """Map POS tag to wordnet POS that lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()

# 2. Lemmatize a word with the appropriate POS tag
stop_words_lemma = set([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in stop_words])

# 3. Add punctuations as stop words
stop_words_lemma.update([i for i in string.punctuation]+["‘", "’", "“", "”"]) 




def statement_score(raw_mask, raw_others, statement, passage):
    """
    raw_mask: list of masked words
    raw_others: list of other tokens in the original sentence
    statement: generated sentence in string
    passage: the passage in string
    """
    
    set_mask = set(raw_mask)
    set_others = set(raw_others) 
    set_statement = set(word_tokenize(statement))
    set_generated = set_statement - set_others
    
    # lemmatization with WordNetLemmatizer
    set_mask = set([lemmatizer.lemmatize(w.lower(), get_wordnet_pos(w.lower())) for w in set_mask])
    set_generated = set([lemmatizer.lemmatize(w.lower(), get_wordnet_pos(w.lower())) for w in set_generated])
    set_passage = set([lemmatizer.lemmatize(w.lower(), get_wordnet_pos(w.lower())) for w in word_tokenize(passage)])
    
    
    # remove stopwords (TODO: try keep stopwords)
    set_mask = set([w for w in set_mask if w not in stop_words_lemma])
    set_generated = set([w for w in set_generated if w not in stop_words_lemma])  
#     print(f"set_mask: {set_mask}")
#     print(f"set_generated: {set_generated}")
    
    # abstractness score
    score_a = 1/(1+math.exp(-0.3*(len(set_mask) - len(set_generated))))
#     print(f"Abstractness score: a={score_a}")
    
    if len(set_generated) == 0:
        print("Blank generated, or the generated text is contained in the original sentence")
#         print(f"Finnal score: 0 \n\n")
        return [0, 0, 0, []]  
        
    set_generated_unique = set_generated - set_mask
#     print(f"set_generated_unique:{set_generated_unique}")
    
    if len(set_generated_unique) == 0:
        print("Same statement as the original sentence is generated.") 
#         print(f"Finnal score: {score_a}\n\n")
        return [score_a, score_a, 0, []]
    
    set_mask_related = set()
    set_mask_WN_cat = set()
    set_mask_Frames = set()
    
    for term in set_mask:
        # conceptnet
        set_mask_related.update(find_related_terms(term, weight=1))
        
#         # wordnet categories,  we only consider verbs and nouns
#         if get_wordnet_pos(term) in [wordnet.VERB, wordnet.NOUN]:
#             for synset in wordnet.synsets(term, pos=get_wordnet_pos(term)):
#                 category = synset.lexname()
#                 set_mask_WN_cat.add(category)
        
        # FrameNet frames
        for f in fn.frames_by_lemma(term):
            set_mask_Frames.add(f.name)
    
#     print(f"set_mask_related: {set_mask_related}")
#     print(f"set_mask_WN_cat: {set_mask_WN_cat}")
#     print(f"set_mask_Frames: {set_mask_Frames}")

    # compute relevance score for the generated statement
    score_r = 0
    norm_term = math.sqrt(len(set_generated_unique))
    relevant_terms = []
    for term in set_generated_unique:
        # lexical or context relevance
        if term in set_passage or term in set_mask_related:
#             print(f"Lexical relevant term:{term}")
            relevant_terms.append(term)
            score_r += 1
            continue
                
        # FrameNet semantic frame relevance
        term_Frames = set()
        for f in fn.frames_by_lemma(term):
            term_Frames.add(f.name)
        if term_Frames & set_mask_Frames:
#             print(f"FrameNet relevant term:{term}, Frame:{term_Frames & set_mask_Frames}")
            relevant_terms.append(term)
            score_r += 1
#             continue
        
#         # WordNet category relevance
#         if get_wordnet_pos(term) in [wordnet.VERB, wordnet.NOUN]:
#             term_WN_cat = set()
#             for synset in wordnet.synsets(term, pos=get_wordnet_pos(term)):
#                 category = synset.lexname()
#                 term_WN_cat.add(category)
#             if term_WN_cat & set_mask_WN_cat:
# #                 print(f"WordNet category relevant term:{term}, category:{term_WN_cat & set_mask_WN_cat}")
#                 relevant_terms.append(term)
#                 score_r += 0.5  # originally 1.0
                
        
    # normalize score
    score_r = score_r / norm_term
#     print(f"set_generated_unique length: {len(set_generated_unique)}")
#     print(f"Total relevance score: {score_r}")
#     print(f"Abstractness score: {score_a}")
#     print(f"Finnal score: {score_r+score_a}\n\n")
    
    return [score_a + score_r, score_a, score_r, relevant_terms]





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package framenet_v17 to /root/nltk_data...
[nltk_data]   Unzipping corpora/framenet_v17.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
raw_mask=['metal','shape']
others=[val for val in sentence.split() if val!='<mask>']
generated=dissimilar[3]
passage=original
scores=statement_score(raw_mask, others, generated, passage)

In [ ]:
scores

[1.0029077523779668, 0.425557483188341, 0.5773502691896258, ['break']]

In [ ]:
print(model)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
from transformers import RobertaForTokenClassification,BertForTokenClassification,AlbertForTokenClassification,ConvBertForTokenClassification
m=ConvBertForTokenClassification.from_pretrained("YituTech/conv-bert-base")
print(m)

Some weights of ConvBertForTokenClassification were not initialized from the model checkpoint at YituTech/conv-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ConvBertForTokenClassification(
  (convbert): ConvBertModel(
    (embeddings): ConvBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ConvBertEncoder(
      (layer): ModuleList(
        (0): ConvBertLayer(
          (attention): ConvBertAttention(
            (self): ConvBertSelfAttention(
              (query): Linear(in_features=768, out_features=384, bias=True)
              (key): Linear(in_features=768, out_features=384, bias=True)
              (value): Linear(in_features=768, out_features=384, bias=True)
              (key_conv_attn_layer): SeparableConv1D(
                (depthwise): Conv1d(768, 768, kernel_size=(9,), stride=(1,), padding=(4,), groups=768, bias=False)
                (pointwise): Conv1d(768, 384, ke

In [ ]:
print(len(Answer_Predictions))

63


In [ ]:
import os
Dataset_folder="/content/gdrive/MyDrive/FYP/Dataset"
for Class in ["Class8","Class9"]:
  for Sub in ["Biology","Geography"]:
    path=os.path.join(Dataset_folder,Class,Sub,"Summary")
    for files in os.listdir(path):
      file_path=os.path.join(path,files)
      a=file_path.split("_")
      print(a[2])

chapter1
chapter2
chapter7
chapter4
chapter5
chapter6
chapter5
chapter6
chapter7
chapter5
chapter6
chapter1
chapter2
chapter3
chapter4
